In [1]:
import time
import math
import copy
import torch
import pickle
import random
import logging
import warnings
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import torch.nn as nn
import torch.optim as opt
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
import fairlearn.datasets as fd
from tqdm import tqdm
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from torch.autograd import grad
from torch.autograd.functional import vhp
from get_datasets import get_diabetes, get_adult, get_law
from torch.utils.data import Subset, DataLoader
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score
from scipy.stats import spearmanr

plt.rcParams['figure.dpi'] = 300
warnings.filterwarnings("ignore")

E = math.e

In [2]:
def get_diabetes():
    #loading Dataset
    data = fd.fetch_diabetes_hospital().data
    data = data.drop(['readmitted'], axis = 1)
    data = data[data.race != 'Unknown'] 
    #data = data[data.max_glu_serum != 'None']
    #data = data[data.A1Cresult != 'None']
    data = data[data.gender != 'Unknown/Invalid'] 
    
    data['change'] = data['change'].replace('Ch', 1)
    data['change'] = data['change'].replace('No', 0)
    
    data['gender'] = data['gender'].replace('Male', 1)
    data['gender'] = data['gender'].replace('Female', 0)
    
    data['diabetesMed'] = data['diabetesMed'].replace('Yes', 1)
    data['diabetesMed'] = data['diabetesMed'].replace('No', 0)


    data['insulin'] = data['insulin'].replace('No', 0)
    data['insulin'] = data['insulin'].replace('Steady', 1)
    data['insulin'] = data['insulin'].replace('Up', 1)
    data['insulin'] = data['insulin'].replace('Down', 1)

    data['A1Cresult'] = data['A1Cresult'].replace('>7', 1)
    data['A1Cresult'] = data['A1Cresult'].replace('>8', 1)
    data['A1Cresult'] = data['A1Cresult'].replace('Norm', 0)
    data['A1Cresult'] = data['A1Cresult'].replace('None', -1)
    
    data['max_glu_serum'] = data['max_glu_serum'].replace('>200', 1)
    data['max_glu_serum'] = data['max_glu_serum'].replace('>300', 1)
    data['max_glu_serum'] = data['max_glu_serum'].replace('Norm', 0)
    data['max_glu_serum'] = data['max_glu_serum'].replace('None', -1)

    data['medicare'] = data['medicare'].replace('True', 1)
    data['medicare'] = data['medicare'].replace('False', 0)
    
    data['medicaid'] = data['medicaid'].replace('True', 1)
    data['medicaid'] = data['medicaid'].replace('False',0)
    
    data['had_emergency'] = data['had_emergency'].replace('True', 1)
    data['had_emergency'] = data['had_emergency'].replace('False',0)
    
    data['had_inpatient_days'] = data['had_inpatient_days'].replace('True', 1)
    data['had_inpatient_days'] = data['had_inpatient_days'].replace('False',0)
    
    data['had_outpatient_days'] = data['had_outpatient_days'].replace('True', 1)
    data['had_outpatient_days'] = data['had_outpatient_days'].replace('False',0)
    
    data['num_lab_procedures'] = data['num_lab_procedures'].apply(lambda x: round(x/10))
    data['num_medications'] = data['num_medications'].apply(lambda x: round(x/10))
    
    
    data = pd.get_dummies(data, columns=['age', 'race', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'medical_specialty','num_lab_procedures', 'num_procedures', 'num_medications', 'primary_diagnosis','number_diagnoses'], drop_first = False)

    return data

In [3]:
 class CreateData(torch.utils.data.Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.targets[idx]

        return out_data, out_label

In [4]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, num_features):
        super(LogisticRegression, self).__init__()
        
        self.fc1 = torch.nn.Linear(num_features, 1, bias=False)
        self.criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
        
    def forward(self, x):
        logits = self.fc1(x)

        return logits
    
    def loss(self, logits, y):
        loss = self.criterion(logits.ravel(), y)
        
        probabilities = torch.sigmoid(logits)
        thresh_results = []
        
        for p in probabilities:
            if p>.5:
                thresh_results.append(1)
            else:
                thresh_results.append(0)
                
        num_correct = 0
        for r,y_ in zip(thresh_results, y):
            if r == y_:
                num_correct += 1
                
        acc = num_correct / len(y) * 100
        prec = precision_score(y.detach().cpu().numpy(), thresh_results, zero_division=0) * 100
        rec = recall_score(y.detach().cpu().numpy(), thresh_results) * 100
        return loss, acc, prec, rec

In [5]:
def train(model, dataset, lr, bs, eps):
    model.train()
    
    opt = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=0)
    criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')

            
    train_data = CreateData(dataset[0], dataset[1])
    train_dataloader = DataLoader(train_data, batch_size=bs, shuffle=True)

    for itr in range(0, eps):
        itr_loss = 0
        for i, [x,y] in enumerate(train_dataloader):
            opt.zero_grad()
            oupt = model(x)
            
            try:
                loss_val = criterion(oupt.ravel(), y)
            except ValueError:
                loss_val = criterion(oupt, y)
            itr_loss += loss_val
            loss_val.backward()
            opt.step() 
        #print(itr_loss/len(train_dataloader))
    return model

In [8]:
data = get_diabetes()
label = 'readmit_binary'

feature_set = set(data.columns) - {label}
num_features = len(feature_set)

print(num_features)

95


In [6]:
def Main(lr, bs, eps):

    device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
    criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
    
    prec_avg = 0
    rec_avg = 0
    acc_avg = 0
    loss_avg = 0
    
    data = get_diabetes()
    label = 'readmit_binary'

    feature_set = set(data.columns) - {label}
    num_features = len(feature_set)

    X = data[feature_set]
    y = data[label]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

    new_train_df = pd.concat([X_train, y_train], axis=1)

    train_sample_num = len(X_train)

    x_test_input = torch.FloatTensor(X_test.values).to(device)
    y_test_input = torch.FloatTensor(y_test.values).to(device)

    x_train_input = torch.FloatTensor(X_train.values).to(device)
    y_train_input = torch.FloatTensor(y_train.values).to(device)

    ##############################################
    # Train original model and get original loss #
    ##############################################
    torch_model = LogisticRegression(num_features)
    torch.save(torch_model.state_dict(), 'models/initial_config.pth')
    torch_model.to(device)
    torch_model = train(torch_model, [x_train_input, y_train_input], lr, bs, eps)
    test_loss, acc, prec, rec = torch_model.loss(torch_model(x_test_input), y_test_input)

    prec_avg += prec
    rec_avg += rec
    acc_avg += acc
    loss_avg += test_loss
        
    return prec_avg, rec_avg, acc_avg, loss_avg

In [7]:
lr = [.0001, .0005, .001, .005, .01, .05, .1, .5]
eps = [10, 15, 20, 25, 50]
bs = [32, 64, 128]

for l in lr:
    for e in eps:
        for b in bs:
            print(f'\nLR: {l}, BS: {b}, Eps: {e}')
            prec, rec, acc, loss = Main(l, b, e)
            print(f'Accuracy: {acc:.2f}')
            print(f'Loss: {loss:.3f}')
            print(f'Precision: {prec:.2f}')
            print(f'Recall: {rec:.2f}')
            


LR: 0.0001, BS: 32, Eps: 10
Accuracy: 54.79
Loss: 0.685
Precision: 52.87
Recall: 29.25

LR: 0.0001, BS: 64, Eps: 10
Accuracy: 54.51
Loss: 0.687
Precision: 51.97
Recall: 33.89

LR: 0.0001, BS: 128, Eps: 10
Accuracy: 53.45
Loss: 0.689
Precision: 50.22
Recall: 34.24

LR: 0.0001, BS: 32, Eps: 15
Accuracy: 57.14
Loss: 0.681
Precision: 57.27
Recall: 32.30

LR: 0.0001, BS: 64, Eps: 15
Accuracy: 56.48
Loss: 0.683
Precision: 55.81
Recall: 32.60

LR: 0.0001, BS: 128, Eps: 15
Accuracy: 53.70
Loss: 0.689
Precision: 50.69
Recall: 30.47

LR: 0.0001, BS: 32, Eps: 20
Accuracy: 58.85
Loss: 0.676
Precision: 60.01
Recall: 35.57

LR: 0.0001, BS: 64, Eps: 20
Accuracy: 57.09
Loss: 0.679
Precision: 56.43
Recall: 35.56

LR: 0.0001, BS: 128, Eps: 20
Accuracy: 55.09
Loss: 0.685
Precision: 52.43
Recall: 41.13

LR: 0.0001, BS: 32, Eps: 25
Accuracy: 58.83
Loss: 0.676
Precision: 59.45
Recall: 37.18

LR: 0.0001, BS: 64, Eps: 25
Accuracy: 56.19
Loss: 0.683
Precision: 56.70
Recall: 26.08

LR: 0.0001, BS: 128, Eps: 25

Accuracy: 61.62
Loss: 0.654
Precision: 60.42
Recall: 51.60

LR: 0.1, BS: 128, Eps: 15
Accuracy: 61.58
Loss: 0.653
Precision: 61.61
Recall: 46.97

LR: 0.1, BS: 32, Eps: 20
Accuracy: 61.62
Loss: 0.658
Precision: 59.12
Recall: 57.69

LR: 0.1, BS: 64, Eps: 20
Accuracy: 61.54
Loss: 0.654
Precision: 61.29
Recall: 47.81

LR: 0.1, BS: 128, Eps: 20
Accuracy: 61.62
Loss: 0.653
Precision: 61.48
Recall: 47.68

LR: 0.1, BS: 32, Eps: 25
Accuracy: 61.52
Loss: 0.655
Precision: 62.47
Recall: 44.04

LR: 0.1, BS: 64, Eps: 25
Accuracy: 61.64
Loss: 0.653
Precision: 61.96
Recall: 46.18

LR: 0.1, BS: 128, Eps: 25
Accuracy: 61.49
Loss: 0.654
Precision: 62.30
Recall: 44.34

LR: 0.1, BS: 32, Eps: 50
Accuracy: 61.22
Loss: 0.659
Precision: 63.56
Recall: 39.73

LR: 0.1, BS: 64, Eps: 50
Accuracy: 61.37
Loss: 0.655
Precision: 62.37
Recall: 43.54

LR: 0.1, BS: 128, Eps: 50
Accuracy: 61.66
Loss: 0.653
Precision: 61.12
Recall: 49.12

LR: 0.5, BS: 32, Eps: 10
Accuracy: 54.48
Loss: 0.747
Precision: 50.72
Recall: 88.14

L